# CS 224W Project: Text Augmented Graphs (OGBN-Products)

### This Colab contains code for experiments using various techniques for combining textual and graph information, tested on the OGBN-Products dataset

## Install PyG and other required libraries

In [ ]:
import argparse
import torch
import torch.nn.functional as F
torch_version = str(torch.__version__)
scatter_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
sparse_src = f"https://pytorch-geometric.com/whl/torch-{torch_version}.html"
!pip install torch-scatter -f $scatter_src
!pip install torch-sparse -f $sparse_src
!pip install torch-geometric
!pip install ogb
!pip install faiss-gpu
import torch_geometric
import torch_geometric.transforms as T
from torch_sparse.tensor import SparseTensor
from torch_geometric.nn import GCNConv, SAGEConv
from ogb.nodeproppred import PygNodePropPredDataset, Evaluator
import numpy as np
import pickle
device = f'cuda' if torch.cuda.is_available() else 'cpu'
device = torch.device(device)

Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html
Looking in links: https://pytorch-geometric.com/whl/torch-2.1.0+cu121.html


## Link the Colab to your Google Drive

We use Google Drive to load the dataset, pre trained LM embeddings and logits. Instructions to download these embeddings are provided later in the Colab


In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


## Add code to load the OGBN Products dataset

Due to the large size of the dataset, and the limited computation available on Colab, we use a sampled version of the OGBN-Products dataset.

The sampled dataset is available at https://github.com/XiaoxinHe/TAPE/blob/main/dataset/ogbn_products/ogbn-products_subset.pt, please create a copy of this file in your Google Drive account and update the filepath in the code accordingly





In [ ]:
DATASET = "ogbn-products"
data_filepath=f'/content/drive/Shareddrives/CS224W Project/ogbn-products_subset.pt'
class LoadData:
    """
    A class used to load and process graph data using the PyTorch Geometric (PyG) library.
    """

    def load_data(self):
        """
        Loads the graph dataset, applies necessary transformations, and retrieves split indices.

        Returns:
        --------
        data: PyG data object
            The transformed graph data.
        split_idx: dict
            A dictionary containing the train, validation, and test split indices.
        num_classes: int
            The number of classes in the dataset.
        """
        num_classes = 47
        data = torch.load(data_filepath)
        data.edge_index = data.adj_t.to_symmetric()
        split_idx={}
        split_idx['train'] = data.train_mask
        split_idx['valid'] = data.val_mask
        split_idx['test'] = data.test_mask
        return data, split_idx, num_classes

## Define the model training and loss function



In [ ]:
class Loss:
    """
    A class for defining loss computation in neural network training.

    """

    def get_loss(self, out, labels, train_idx):
        """
        Computes the cross-entropy loss between the output predictions and the true labels.

        Parameters:
        -----------
        out : torch.Tensor
            The output predictions from the neural network model, typically the logits.
        labels : torch.Tensor
            The true labels for the training data.
        train_idx : torch.Tensor or list
            The indices of the training data samples.

        Returns:
        --------
        torch.Tensor
            The computed cross-entropy loss.
        """
        return F.cross_entropy(out, labels[train_idx])


def train(model, data, train_idx, optimizer, loss_obj):
    """
    Trains a neural network model for one epoch.

    Parameters:
    -----------
    model : torch.nn.Module
        The neural network model to be trained.
    data : object
        The data object containing features and adjacency information.
    train_idx : torch.Tensor or list
        The indices of the training data.
    optimizer : torch.optim.Optimizer
        The optimizer used for updating model weights.
    loss_obj : Loss
        An instance of the Loss class to compute the loss.

    Returns:
    --------
    float
        The loss value computed for this training epoch.
    """
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index, train_idx)[train_idx]
    loss = loss_obj.get_loss(out, data.y.squeeze(1), train_idx)
    loss.backward()
    optimizer.step()
    return loss.item()

## Define the model testing function


In [ ]:
@torch.no_grad()
def test(model, data, split_idx, evaluator):
    """
    Evaluates the performance of a trained model on training, validation, and test datasets.

    Parameters:
    -----------
    model : torch.nn.Module
        The neural network model to be evaluated.
    data : object
        The data object containing features, adjacency information, and labels.
    split_idx : dict
        A dictionary with keys 'train', 'valid', and 'test' mapping to the respective data indices.
    evaluator : object
        An object used to evaluate the model's predictions. It must have an 'eval' method.

    Returns:
    --------
    tuple: (train_acc, valid_acc, test_acc)
        A tuple containing the accuracy on the training, validation, and test sets.
    """
    model.eval()

    out = model(data.x, data.edge_index)
    y_pred = out.argmax(dim=-1, keepdim=True).cpu()

    # Evaluate accuracy on training, validation, and test sets
    train_acc = evaluator.eval({
        'y_true': data.y[split_idx['train']],
        'y_pred': y_pred[split_idx['train']],
    })['acc']
    valid_acc = evaluator.eval({
        'y_true': data.y[split_idx['valid']],
        'y_pred': y_pred[split_idx['valid']],
    })['acc']
    test_acc = evaluator.eval({
        'y_true': data.y[split_idx['test']],
        'y_pred': y_pred[split_idx['test']],
    })['acc']

    return train_acc, valid_acc, test_acc


## Define Hyperparameters

Our code support two models


1.   GraphSAGE (torch_geometric.nn.models.GraphSAGE)
2.   GCN (torch_geometric.nn.models.GCN)




In [ ]:
hyperparams = {'model' : 'GraphSAGE', 'hidden_layer_size' : 128, 'num_layers' : 3, 'dropout' : 0.5, 'learning_rate' : 1e-3, 'epochs' : 1000}

## Create an end-to-end training loop



In [ ]:
def train_loop(load_data_obj, loss_obj, hyperparams):
  """
    Executes the training loop for a graph neural network using specified hyperparameters.


    Parameters:
    -----------
    load_data_obj : LoadData
        An instance of the LoadData class that is used to load the dataset.
    loss_obj : Loss
        An instance of the Loss class that defines the loss function to be used during training.
    hyperparams : dict
        A dictionary containing hyperparameters for the model. Expected keys are:
        'model' (str), 'hidden_layer_size' (int), 'num_layers' (int), 'out_channels' (int),
        'dropout' (float), 'learning_rate' (float), 'epochs' (int).

    Returns:
    --------
    tuple:
        best_test_acc (float): The highest test accuracy achieved during training.
        model: The trained model instance.
  """
  data, split_idx, num_classes = load_data_obj.load_data()
  if hyperparams['model'] == 'GraphSAGE':
    model = torch_geometric.nn.models.GraphSAGE(data.x.shape[1], hidden_channels=hyperparams['hidden_layer_size'],
                                              num_layers=hyperparams['num_layers'], out_channels=num_classes, dropout=hyperparams['dropout']).to(device)
  else:
    model = torch_geometric.nn.models.GCN(data.x.shape[1], hidden_channels=hyperparams['hidden_layer_size'],
                                              num_layers=hyperparams['num_layers'], out_channels=num_classes, dropout=hyperparams['dropout']).to(device)
  optimizer = torch.optim.Adam(model.parameters(), lr=hyperparams['learning_rate'])
  evaluator = Evaluator(name=DATASET)
  train_idx = split_idx['train']
  model.to(device)
  data.to(device)
  valid_accs=[]
  test_accs=[]
  for epoch in range(hyperparams['epochs']):
    loss = train(model, data, train_idx, optimizer, loss_obj)
    result = test(model, data, split_idx, evaluator)
    train_acc, valid_acc, test_acc = result
    valid_accs.append(valid_acc)
    test_accs.append(test_acc)
    print(f'Epoch: {epoch:02d}, '
          f'Loss: {loss:.4f}, '
          f'Train: {100 * train_acc:.2f}%, '
          f'Valid: {100 * valid_acc:.2f}% '
          f'Test: {100 * test_acc:.2f}%')
  best_test_acc = test_accs[np.argmax(valid_accs)]
  print(f'Best Test accuracy is {best_test_acc}')
  return best_test_acc, model

## Run the training loop using the Base Model

In [ ]:
load_data=LoadData()
loss_obj=Loss()
standard_acc, base_model = train_loop(load_data, loss_obj, hyperparams)

Epoch: 00, Loss: 3.8940, Train: 15.74%, Valid: 14.95% Test: 9.70%
Epoch: 01, Loss: 3.7362, Train: 28.39%, Valid: 25.51% Test: 17.91%
Epoch: 02, Loss: 3.5971, Train: 29.33%, Valid: 26.46% Test: 18.44%
Epoch: 03, Loss: 3.4508, Train: 28.30%, Valid: 25.19% Test: 17.81%
Epoch: 04, Loss: 3.3230, Train: 27.65%, Valid: 24.49% Test: 17.50%
Epoch: 05, Loss: 3.1815, Train: 27.45%, Valid: 24.36% Test: 17.41%
Epoch: 06, Loss: 3.0695, Train: 27.36%, Valid: 24.36% Test: 17.39%
Epoch: 07, Loss: 2.9579, Train: 27.35%, Valid: 24.36% Test: 17.37%
Epoch: 08, Loss: 2.8536, Train: 27.36%, Valid: 24.36% Test: 17.39%
Epoch: 09, Loss: 2.7652, Train: 27.47%, Valid: 24.43% Test: 17.44%
Epoch: 10, Loss: 2.6899, Train: 28.05%, Valid: 24.94% Test: 17.64%
Epoch: 11, Loss: 2.6173, Train: 29.47%, Valid: 26.02% Test: 18.16%
Epoch: 12, Loss: 2.5642, Train: 31.15%, Valid: 27.67% Test: 18.90%
Epoch: 13, Loss: 2.4942, Train: 32.54%, Valid: 29.77% Test: 19.60%
Epoch: 14, Loss: 2.4255, Train: 34.11%, Valid: 31.36% Test: 20.

# Load the dataset with LM initialized embeddings

We finetune an MPNet model on the ogbn-products task using the web page text as an input feature. We then use the embeddings generated by this model as our initial node features for the GNN.

The embeddings are available at https://drive.google.com/file/d/184qquWQuXbSog2PDZMG5xuWZU043hn3u/view?usp=sharing, please create a copy of this file in your Google Drive account and update the filepath in the code accordingly



In [ ]:
filepath= "/content/drive/Shareddrives/CS224W Project/products_embeddings_logits/mpnet_products.pkl"
embs = pickle.load(open(filepath, "rb"))


In [ ]:
class LoadDataLMInit(LoadData):
    """
    A class extending LoadData to load graph data with initial node embeddings.

    Attributes:
    -----------
    embs : array-like
        An array of node embeddings used to initialize the node features in the graph dataset.
    """

    def __init__(self, embs):
        """
        The constructor for LoadDataLMInit class.

        Initializes the LoadDataLMInit instance with the provided node embeddings.

        Parameters:
        -----------
        embs : array-like
            An array-like structure containing node embeddings. Each element in the array
            represents the embedding of a node in the graph.
        """
        self.embs = embs
        super(LoadDataLMInit, self).__init__()

    def load_data(self):
        """
        Loads the graph dataset with initial node embeddings, applies transformations, and retrieves split indices.

        Returns:
        --------
        data : PyG data object
            The graph data object with initialized node features.
        split_idx : dict
            A dictionary containing the indices for train, validation, and test splits.
        num_classes : int
            The number of classes in the dataset.
        """
        num_classes = 47
        data = torch.load(data_filepath)
        data.edge_index = data.adj_t.to_symmetric()
        split_idx={}
        split_idx['train'] = data.train_mask
        split_idx['valid'] = data.val_mask
        split_idx['test'] = data.test_mask
        data.x = torch.tensor(self.embs)
        return data, split_idx, num_classes

## Run the pipeline with LM initialized embeddings

In [ ]:
load_data=LoadDataLMInit(embs)
lminit_acc, lminit_model = train_loop(load_data, loss_obj, hyperparams)

Epoch: 00, Loss: 3.8677, Train: 53.65%, Valid: 50.57% Test: 40.10%
Epoch: 01, Loss: 3.6938, Train: 71.25%, Valid: 70.10% Test: 55.62%
Epoch: 02, Loss: 3.5192, Train: 77.94%, Valid: 77.74% Test: 62.01%
Epoch: 03, Loss: 3.2925, Train: 80.91%, Valid: 81.11% Test: 63.83%
Epoch: 04, Loss: 3.0215, Train: 81.15%, Valid: 81.62% Test: 63.89%
Epoch: 05, Loss: 2.7082, Train: 80.61%, Valid: 81.49% Test: 63.71%
Epoch: 06, Loss: 2.3675, Train: 79.66%, Valid: 80.03% Test: 63.16%
Epoch: 07, Loss: 2.0437, Train: 78.80%, Valid: 78.94% Test: 62.46%
Epoch: 08, Loss: 1.7587, Train: 78.35%, Valid: 78.37% Test: 62.09%
Epoch: 09, Loss: 1.5318, Train: 78.31%, Valid: 78.37% Test: 62.09%
Epoch: 10, Loss: 1.3633, Train: 78.95%, Valid: 79.01% Test: 62.55%
Epoch: 11, Loss: 1.2317, Train: 80.24%, Valid: 80.73% Test: 63.57%
Epoch: 12, Loss: 1.1388, Train: 82.09%, Valid: 82.89% Test: 64.69%
Epoch: 13, Loss: 1.0707, Train: 82.87%, Valid: 83.46% Test: 65.21%
Epoch: 14, Loss: 1.0114, Train: 83.33%, Valid: 84.10% Test: 65

## Add virtual edges to the Dataset

We attempt data augmentation by adding virtual edges, to capture relationships between nodes that have similar titles and abstracts. We add edges between each node and its k nearest neighbours based on the LLM embeddings of the MPNet model explained above. We use the [FAISS](https://github.com/facebookresearch/faiss) library for fast calculation of nearest neighbours.

In [ ]:
import faiss
class LoadDataVirtualEdges(LoadData):
  """
    A class extending LoadData to load graph data with additional virtual edges.

    Attributes:
    -----------
    embs : array-like
        An array of node embeddings used to find nearest neighbors and create virtual edges.
    k : int
        The number of nearest neighbors to consider for creating virtual edges.
  """

  def __init__(self, embs, k):
    self.embs=embs
    self.k=k
    super(LoadDataVirtualEdges, self).__init__()

  def get_nearest_neighbours(self, embs, k):
      """
      Computes the k-nearest neighbors for each node in the graph based on embeddings.

      Parameters:
      -----------
      embs : array-like
          An array-like structure containing node embeddings.
      k : int
          The number of nearest neighbors to find for each node.

      Returns:
      --------
      numpy.ndarray
          An array of indices representing the k-nearest neighbors for each node.
      """
      res = faiss.StandardGpuResources()
      index = faiss.IndexFlatL2(embs.shape[1])   # build the index
      gpu_index_flat = faiss.index_cpu_to_gpu(res, 0, index)
      gpu_index_flat.add(embs)                  # add vectors to the index
      D, I = gpu_index_flat.search(embs, k+1)     # actual search
      return I

  def convert_to_edge_index(self, I, k):
    """
    Converts nearest neighbor information into PyTorch Geometric edge indices.

    Parameters:
    -----------
    I : numpy.ndarray
        An array of indices representing the nearest neighbors for each node.
    k : int
        The number of nearest neighbors for each node.

    Returns:
    --------
    tuple: (src_nodes, dst_nodes)
        Two tensors representing the source and destination nodes of each virtual edge.
    """
    num_nodes = I.shape[0]
    src_nodes = torch.arange(num_nodes).repeat_interleave(k).to(device)
    dst_nodes = torch.tensor(I[:,1:]).flatten().to(device)
    return src_nodes, dst_nodes

  def load_data(self):
    """
    Loads the graph dataset, adds virtual edges based on nearest neighbors, applies transformations,
    and retrieves split indices.

    Returns:
    --------
    data : PyG data object
        The graph data object with virtual edges added.
    split_idx : dict
        A dictionary containing the indices for train, validation, and test splits.
    num_classes : int
        The number of classes in the dataset.
    """
    num_classes = 47
    data = torch.load(data_filepath)
    row, col, edge_attr = data.adj_t.to_symmetric().t().coo()
    data.edge_index = torch.stack([row, col], dim=0)
    split_idx={}
    split_idx['train'] = data.train_mask
    split_idx['valid'] = data.val_mask
    split_idx['test'] = data.test_mask
    I=self.get_nearest_neighbours(self.embs,self.k)
    S,D=self.convert_to_edge_index(I,k)
    data.edge_index=torch.cat((data.edge_index,torch.stack((S,D)).to('cpu')),dim=1)
    data.edge_index=SparseTensor.from_edge_index(data.edge_index)
    return data, split_idx, num_classes

## Run the pipeline with Virtual Edges added

In [ ]:
k=10
load_data=LoadDataVirtualEdges(embs,k)
virtual_edge_acc, virtual_edge_model = train_loop(load_data, loss_obj, hyperparams)

Epoch: 00, Loss: 3.8834, Train: 10.14%, Valid: 10.18% Test: 6.94%
Epoch: 01, Loss: 3.7166, Train: 26.49%, Valid: 23.92% Test: 17.03%
Epoch: 02, Loss: 3.5556, Train: 27.44%, Valid: 24.36% Test: 17.53%
Epoch: 03, Loss: 3.4017, Train: 27.33%, Valid: 24.36% Test: 17.43%
Epoch: 04, Loss: 3.2464, Train: 27.31%, Valid: 24.36% Test: 17.39%
Epoch: 05, Loss: 3.0988, Train: 27.31%, Valid: 24.36% Test: 17.38%
Epoch: 06, Loss: 2.9483, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 07, Loss: 2.8056, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 08, Loss: 2.6832, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 09, Loss: 2.5648, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 10, Loss: 2.4675, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 11, Loss: 2.3909, Train: 27.44%, Valid: 24.49% Test: 17.46%
Epoch: 12, Loss: 2.2997, Train: 29.25%, Valid: 26.21% Test: 18.53%
Epoch: 13, Loss: 2.2167, Train: 33.74%, Valid: 30.92% Test: 21.25%
Epoch: 14, Loss: 2.1476, Train: 39.45%, Valid: 35.88% Test: 24.

## Add KL Divergence Regularization

KLLoss extends the basic loss computation (like cross-entropy) by adding a regularization term
    based on KL divergence. This is useful in scenarios where one wishes to penalize the divergence
    between the model's output distribution and a target distribution, which in this case is provided
    by 'lm_logits'. The lambda (lmbda) parameter controls the weight of this regularization term.


The MPNET logits are available at https://drive.google.com/file/d/18gitCCIT5f_hS1p-CcDQB1a5djWVSgV-/view?usp=drive_link, please create a copy of this file in your Google Drive account and update the filepath in the code accordingly


In [ ]:
filepath_logits='/content/drive/Shareddrives/CS224W Project/products_embeddings_logits/mpnet_logits_products.pkl'
lm_logits=pickle.load(open(filepath_logits, "rb"))

In [ ]:
class KLLoss(Loss):
    """
    A class extending Loss to incorporate Kullback-Leibler (KL) divergence as a regularization term.

    Attributes:
    -----------
    lm_logits : torch.Tensor
        The logits from a language model or a pre-defined target distribution.
    lmbda : float
        The weight (lambda) of the KL divergence regularization term in the overall loss.
    """

    def __init__(self, lm_logits, lmbda):
        """
        The constructor for KLLoss class.

        Initializes the KLLoss instance with provided language model logits and lambda value.

        Parameters:
        -----------
        lm_logits : torch.Tensor
            The logits from a language model or a pre-defined target distribution.
        lmbda : float
            The weight (lambda) of the KL divergence regularization term in the overall loss.
        """
        self.lmbda = lmbda
        self.lm_logits = lm_logits
        super(KLLoss, self).__init__()

    def get_loss(self, out, labels, train_idx):
        """
        Parameters:
        -----------
        out : torch.Tensor
            The output predictions from the neural network model, typically the logits.
        labels : torch.Tensor
            The true labels for the training data.
        train_idx : torch.Tensor or list
            The indices of the training data samples.

        Returns:
        --------
        torch.Tensor
            The computed loss, combining cross-entropy and KL divergence.
        """
        loss = F.cross_entropy(out, labels[train_idx])
        reg_penalty = torch.nn.KLDivLoss(log_target=True)
        reg = reg_penalty(F.log_softmax(out, dim=1), F.log_softmax(torch.tensor(self.lm_logits[train_idx]).to(device), dim=1))
        loss += reg * self.lmbda
        return loss


In [ ]:
lmbda=1
load_data=LoadData()
kl_loss_obj=KLLoss(lm_logits,lmbda)
kl_best_acc = train_loop(load_data, kl_loss_obj, hyperparams)

/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:2943: UserWarning: reduction: 'mean' divides the total loss by both the batch size and the support size.'batchmean' divides only by the batch size, and aligns with the KL div math definition.'mean' will be changed to behave the same as 'batchmean' in the next major release.
  warnings.warn(


Epoch: 00, Loss: 3.9299, Train: 15.34%, Valid: 14.25% Test: 9.66%
Epoch: 01, Loss: 3.7758, Train: 27.05%, Valid: 24.05% Test: 16.93%
Epoch: 02, Loss: 3.6415, Train: 27.55%, Valid: 24.62% Test: 17.45%
Epoch: 03, Loss: 3.5012, Train: 27.41%, Valid: 24.43% Test: 17.41%
Epoch: 04, Loss: 3.3658, Train: 27.33%, Valid: 24.43% Test: 17.38%
Epoch: 05, Loss: 3.2378, Train: 27.33%, Valid: 24.36% Test: 17.37%
Epoch: 06, Loss: 3.1191, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 07, Loss: 3.0035, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 08, Loss: 2.9094, Train: 27.31%, Valid: 24.36% Test: 17.37%
Epoch: 09, Loss: 2.8163, Train: 27.32%, Valid: 24.36% Test: 17.37%
Epoch: 10, Loss: 2.7386, Train: 27.37%, Valid: 24.43% Test: 17.38%
Epoch: 11, Loss: 2.6790, Train: 27.73%, Valid: 24.81% Test: 17.52%
Epoch: 12, Loss: 2.6059, Train: 28.92%, Valid: 25.76% Test: 18.02%
Epoch: 13, Loss: 2.5443, Train: 31.17%, Valid: 27.67% Test: 18.95%
Epoch: 14, Loss: 2.4885, Train: 33.55%, Valid: 29.96% Test: 20.

## Ensembling

We demonstrate an ensemble approach combining a base graph neural network model with scaled logits from a language model.



In [ ]:
data,split_idx,_=LoadData().load_data()
data.to(device)
base_model.to(device)
final_out = base_model(data.x, data.adj_t)
lm_logits = torch.tensor(lm_logits).to(device)
max_test_acc=0
max_i=0
for i in [1,5,10,15,20]:
  final_out = final_out + i*lm_logits
  y_pred = final_out.argmax(dim=-1, keepdim=True)
  evaluator = Evaluator(name=DATASET)
  test_acc = evaluator.eval({
          'y_true': data.y[split_idx['test']],
          'y_pred': y_pred[split_idx['test']],
      })['acc']
  if test_acc>max_test_acc:
    max_test_acc=test_acc
    max_i=i
print(f'Max test acc: {max_test_acc}')

Max test acc: 0.7417936150483507
